In [1]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
print(my_address)
target_amount = 0.1
fee = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(my_address))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = '''mncy8Ly8BK2t6tM8aYZEBPRhbwsZD8KxY9
mrckabL9GWjcGytQg1NRvDQat45Umk9mj8
moLdtNAPFZoFN68q5psEg6iHTSezWLvna2
mqiD6KF3CUwX4s6cS2Py4UakqTdhZkhUnT
mu9JroK2CCPQXGtQGynsa8Dd3B7gyrK6Gv
mnRrSqSYmsDAgwmjPiz77HJfz1XsfKM3WW
mix6nFZyiiTwFFhqhQuzUYduhSvYAkMEXN
mwQSGx7tQxPU6ECGBhxPQ4uofsV5RestnZ'''
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['tx_hash_big_endian'])
    prev_index = utxo['tx_output_n']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = int(target_amount * 100000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_satoshis,
        script_pubkey=script_pubkey,
    ))

change_satoshis = total - int(fee*100000000) - output_amount
h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv
010000000112e497ade6b439d8f7ff9ebd124c964721c632e3168b697936469d28c8689ed8010000006b483045022100a442efd7619acaf0b11d467298adee0d74189e23243bba00771454f5a9598e5702200394e08da906e2cdb2bcb089b761c399210663ef9bf3963cc1e4ba40bb5f16b10121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6effffffff0980969800000000001976a9144dec123f6ffd6e37ba81c0d7f2dc0ce28316403388ac80969800000000001976a91479c219af167afa8ef4ee92b711697426dd23aaaa88ac80969800000000001976a91455cd7ae659b93addeae329b421afacf621c8a0ae88ac80969800000000001976a9146fd22f93297b5250b18eaadf4b9d1defe5a7d0d888ac80969800000000001976a9149579c6b8a3d98a1b51cc6fb92524da62f1075b1688ac80969800000000001976a9144bd1eb2457091bf1a4157291a2c11093628f666888ac80969800000000001976a91425a9dc3c576c264ca2bd297e0b2600e4da1ab18488ac80969800000000001976a914ae467549d3c0105a6a4632dd58cc85e9d2d931dd88aca6290992000000001976a9146e13971913b9aa89659a9f53d327baa8826f2d7588ac00000000


In [ ]:
tx_in = TxIn(
    bytes.fromhex(
        'e591ea6abf3db4812e6ba594a5015a079bb40b5962414367e210ff5ac0c62826'), 1)
amount = tx_in.value(testnet=True) - 50000
script_pubkey = p2pkh_script(decode_base58('mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'))
print(script_pubkey)
tx_out = TxOut(amount, script_pubkey)

tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
print(tx_obj.serialize().hex())

In [ ]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'

secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
target_amount = 0.1
fee = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(my_address))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['tx_hash_big_endian'])
    prev_index = utxo['tx_output_n']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))

# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = randint(5000000, 20000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(target_satoshis, script_pubkey))

change_satoshis = total - int((fee)*100000000) - output_amount

h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

In [13]:
from ecc import S256Point, Signature, G, N
from random import randint

point = S256Point.parse(bytes.fromhex('04678afdb0fe5548271967f1a67130b7105cd6a828e03909a67962e0ea1f61deb649f6bc3f4cef38c4f35504e51ec112de5c384df7ba0b8d578a4c702b6bf11d5f'))

u = 1
v = 1
r = (u*G+v*point).x.num % N
s = r * pow(v, N-2, N) % N
z = u * s % N

sig = Signature(r,s)
print(point.verify(z, sig))
print(hex(z))
print(point.sec(compressed=False).hex())
print(sig.der().hex())

True
0xd20a8b8b4d25086d71f03358d26d8564fc199aefd2a0239c49e9ab4c93a7025e
04678afdb0fe5548271967f1a67130b7105cd6a828e03909a67962e0ea1f61deb649f6bc3f4cef38c4f35504e51ec112de5c384df7ba0b8d578a4c702b6bf11d5f
3046022100d20a8b8b4d25086d71f03358d26d8564fc199aefd2a0239c49e9ab4c93a7025e022100d20a8b8b4d25086d71f03358d26d8564fc199aefd2a0239c49e9ab4c93a7025e


In [10]:
from helper import hash256

print(hash256(b'The Times 19/11/2018 The ups and downs of Downing Street').hex())

2e1d1cc2a4ca52c6f6178570da8375365bc06416b898eb9436f328a4eb72d22d


In [15]:
M = 0xc8b0ce42f46f4ec0517fcd68f56776123031e00cd6f67625f93ddef0d96a3313
R = 0xd87d8097c920245f037f2a134ad2eb1e551aa1a3d7cb5a3e38f7353890f46d38
S = 0x27827f6836dfdba0fc80d5ecb52d14e065943b42d77d45fd86db29543f41d409
X = 0x11db93e1dcdb8a016b49840f8c53bc1eb68a382e97b1482ecad7b148a6909a5c
Y = 0xb2e0eaddfb84ccf9744464f82e160bfa9b8b64f9d4c03f999b8643f656b412a3
print(R+S == N)
print(hex(M * pow(S, N-2,N) % N))

True
0xcdfa8e0aa397127e5fd4757d9fa63f7b45c6c4eac21fc4938ca23fddb6a3b39d


In [20]:
from ecc import Point, FieldElement

prime = 199
a = FieldElement(0, prime)
b = FieldElement(7, prime)

for x_raw in range(prime):
    x = FieldElement(x_raw, prime)
    for y_raw in range(prime):
        y = FieldElement(y_raw, prime)
        if y**2 == x**3 + a*x + b:
            print("{}\t{}".format(x_raw, y_raw))


0	87
0	112
1	40
1	159
5	27
5	172
7	56
7	143
8	11
8	188
9	61
9	138
11	12
11	187
16	83
16	116
17	12
17	187
19	10
19	189
20	68
20	131
21	78
21	121
23	45
23	154
24	10
24	189
27	63
27	136
32	61
32	138
33	70
33	129
37	78
37	121
40	39
40	160
46	80
46	119
47	56
47	143
49	68
49	131
50	45
50	154
51	70
51	129
52	11
52	188
53	80
53	119
54	96
54	103
55	60
55	139
57	54
57	145
59	60
59	139
60	93
60	106
61	39
61	160
62	27
62	172
67	9
67	190
70	54
70	145
71	64
71	135
72	54
72	145
73	24
73	175
75	20
75	179
76	63
76	136
79	83
79	116
82	44
82	155
84	41
84	158
85	60
85	139
88	74
88	125
90	95
90	104
92	40
92	159
95	17
95	182
96	63
96	136
97	71
97	128
98	39
98	160
100	80
100	119
103	5
103	194
104	83
104	116
105	7
105	192
106	40
106	159
108	7
108	192
115	70
115	129
120	17
120	182
121	95
121	104
123	5
123	194
126	45
126	154
130	68
130	131
132	27
132	172
133	71
133	128
134	20
134	179
135	44
135	155
136	74
136	125
137	9
137	190
139	11
139	188
141	78
141	121
145	56
145	143
147	93
147	106
148	41
148	158
149	24
149